In [21]:
# Let's creat a random kitchen sink model

# Instead I might use https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html

# Import the necessary libraries
import numpy as np

def get_iid_gaussian(d, number_of_samples=1, dimension_adjusted_variance = False ):
    """
    returns a (n,d) dimensional normal sample

    d: dimension
    number_of_samples: number of d dimensional samples to produce
    dimension_adjusted_variance: divide variance by dimension
    """
    variance_term = 1 if not dimension_adjusted_variance else d
    return np.random.normal(0.,1./np.sqrt(variance_term),size=(number_of_samples,d))

def sample_weights(d):
    """
    returns a sample weight of dimension (d,)
    Weights are iid gaussians with mean 0 and variance 1

    d: dimension
    """
    return np.reshape(get_iid_gaussian(d), (d,))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Load the sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Sample the mean of the sigmoid from a normal distributino
def sample_k0(n):
    return np.mean(sigmoid(np.random.normal(0, 1, n)))

def sample_k1(n):
    z = np.random.normal(0, 1, n)
    return np.mean(sigmoid(z) * z)

def sample_k2(n, k0, k1):
    return np.sqrt( np.mean(sigmoid(np.random.normal(0,1,n)**2)) - k0**2 - k1**2 )

n = 1000000

k0 = sample_k0(n)
k1 = sample_k1(n)
k2 = sample_k2(n, k0, k1)

print(k0, k1, k2)

0.4998483787508702 0.20535118875055666 0.6128822611695713


In [23]:
# Let's sample the random matrix F
p = 1000
d = 800

# F*F^T needs to be of dimension dxd
# Hence, F needs to be of dimension dxp
F = np.random.normal(0, 1, (d, p))

print(F.shape)

(800, 1000)


In [26]:
# Let's sample teacher weights

# The teacher weights are of dimension p
theta = sample_weights(p)

Psi = np.eye(p)
Phi = k1 * F.T
Omega = k0**2 * np.ones(d) * np.ones(d).T + k1**2 * F @ F.T/d + k2**2 * np.eye(d)

print(Psi.shape, Phi.shape, Omega.shape)

(1000, 1000) (1000, 800) (800, 800)
